# PyTerrier ECIR 2021 Tutorial Notebook - Part 3.1 - OpenNIR and monoT5

This is one of a series of Colab notebooks created for This notebook provides experiences to attendees for creating indexing pipelines in PyTerrier. All experiments are conducted using the CORD19 corpus and the TREC Covid test collection.

This notebook aims to demonstrate use of the neural re-rankers using the [OpenNIR](https://opennir.net/) and [monoT5](https://github.com/terrierteam/pyterrier_t5) plugins.

In this notebook, you will experience:

 - re-ranking documents using neural models like KNRM, Vanilla BERT, EPIC, and monoT5.


## Setup

In the following, we will set up the libraries required to execute the notebook.

### Pyterrier installation

The following cell installs the latest release of the [PyTerrier](https://github.com/terrier-org/pyterrier) package.

In [ ]:
!pip install --upgrade python-terrier

### Pyterrier plugins installation 

We install the official version of the [OpenNIR](https://opennir.net/) and [monoT5](https://github.com/terrierteam/pyterrier_t5) plugins. PyTerrier plugins. You can safely ignore the package versioning errors.

In [ ]:
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

## Preliminary steps

### [PyTerrier](https://github.com/terrier-org/pyterrier) initialization 

Lets get [PyTerrier](https://github.com/terrier-org/pyterrier) started. This will download the latest version of the [Terrier](http://terrier.org/) IR platform. We also import the [OpenNIR](https://opennir.net/) pyterrier bindings.

In [ ]:
import pyterrier as pt
if not pt.started():
    pt.init(tqdm='notebook')
import onir_pt

### [TREC-COVID19](https://ir.nist.gov/covidSubmit/) Dataset download

The following cell downloads the [TREC-COVID19](https://ir.nist.gov/covidSubmit/) dataset that we will use in the remainder of this notebook.

In [ ]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = dataset.get_topics(variant='description')
qrels = dataset.get_qrels()

### Terrier inverted index download

To save a few minutes, we use a pre-built Terrier inverted index for the TREC-COVID19 collection. Download time took a few seconds for us.

The construction of the inverted index will take few minutes, and the code to use is the following:

```python
import os

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)
```

In [ ]:
import os

if not os.path.exists("terrier_index.zip"):
  !wget http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip
  !unzip -j terrier_index.zip -d terrier_index

index_ref = pt.IndexRef.of("./terrier_index/data.properties")
index = pt.IndexFactory.of(index_ref)

## Re-Rankers from scratch

Let's start exploring a few neural re-ranking methods! We can build them from scratch using `onir_pt.reranker`.

And OpenNIR reranking model consists of:
 - `ranker` (e.g., `drmm`, `knrm`, or `pacrr`). This defines the neural ranking architecture.
 - `vocab` (e.g., `wordvec_hash`, or `bert`). This defines how text is encoded by the model. This approach makes it easy to swap out different text representations.

In [ ]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

Let's look at how well these models work at ranking!

In [ ]:
br = pt.BatchRetrieve(index) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

This doesn't work very well because the model is not trained; it's using random weights to combine the scores from the similarity matrix.

## Loading a trained re-ranker

You can train re-ranking models in PyTerrier using the `fit` method. This takes a bit of time, so we'll download a model that's already been trained. If you'd like to train the model yourself, you can use:

```python
# transfer training signals from a medical sample of MS MARCO
from sklearn.model_selection import train_test_split
train_ds = pt.datasets.get_dataset('irds:msmarco-passage/train/medical')
train_topics, valid_topics = train_test_split(train_ds.get_topics(), test_size=50, random_state=42) # split into training and validation sets

# Index MS MARCO
indexer = pt.index.IterDictIndexer('./terrier_msmarco-passage')
tr_index_ref = indexer.index(train_ds.get_corpus_iter(), fields=('text',), meta=('docno',))

pipeline = (pt.BatchRetrieve(tr_index_ref) % 100 # get top 100 results
            >> pt.text.get_text(train_ds, 'text') # fetch the document text
            >> pt.apply.generic(lambda df: df.rename(columns={'text': 'abstract'}) # rename columns
            >> knrm) # apply neural re-ranker

pipeline.fit(
    train_topics,
    train_ds.get_qrels(),
    valid_topics,
    train_ds.get_qrels())
```

In [ ]:
del knrm # free up the memory before loading a new version of the ranker
knrm = onir_pt.reranker.from_checkpoint('https://macavaney.us/knrm.medmarco.tar.gz', text_field='abstract', expected_md5="d70b1d4f899690dae51161537e69ed5a")

In [ ]:
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

That's a little better than before, but it still underperforms our first-stage ranking model.

## Vanilla BERT

Contextualized language models, such as [BERT](https://arxiv.org/abs/1810.04805), are much more powerful neural models that have been shown to be effective for ranking.

We'll try using a "vanilla" (or "mono") version of the BERT model. The BERT model is pre-trained for the task of language modeling and next sentence prediction.

In [ ]:
del knrm # clear out memory from KNRM
vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='abstract', vocab_config={'train': True})

Let's see how this model does on TREC COVID.

In [ ]:
pipeline = br % 100 >> pt.text.get_text(dataset, 'abstract') >> vbert
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> VBERT'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

As we see, although the model is pre-trained, it doesn't do very well at ranking on our benchmark. This is because it's not tuned for the task of relevance ranking.

We can train the model for ranking (as shown above for KNRM) or we can download a trained model. Here, we use the [SLEDGE](https://arxiv.org/abs/2010.05987) model, which is a Vanilla BERT model trained on scientific text and tuned on medical queries.

In [ ]:
vbert = onir_pt.reranker.from_checkpoint('https://macavaney.us/scibert-medmarco.tar.gz', text_field='abstract', expected_md5="854966d0b61543ffffa44cea627ab63b")

In [ ]:
pipeline = br % 100 >> pt.text.get_text(dataset, 'abstract') >> vbert
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> VBERT'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

That's much better! We're able to improve upon the first stage ranker. But we can see that this is pretty slow to run.

## EPIC

Some models focus on query-time computational efficiency. The [EPIC](https://arxiv.org/abs/2004.14245) model builds light-weight document representations that are independent of the query. This means that they can be computed ahead of time. You can index the corpus yourself with the following code (but it takes a while):

```python
indexed_epic = onir_pt.indexed_epic.from_checkpoint('https://macavaney.us/epic.msmarco.tar.gz', index_path='./epic_cord19')
indexed_epic.index(dataset.get_corpus_iter(), fields=('abstract',))
```

Instead, we'll download a copy of the EPIC-processed documents:

In [ ]:
import os
if not os.path.exists('epic_cord19.zip'):
  !wget http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/epic_cord19.zip
  !unzip epic_cord19.zip
indexed_epic = onir_pt.indexed_epic.from_checkpoint('https://macavaney.us/epic.msmarco.tar.gz', index_path='./epic_cord19')

We can now run this model over the results of a first-stage ranker. Note how we do not need to fetch the document text with `pt.text.get_text`, which further saves time.

In [ ]:
br = pt.BatchRetrieve(index) % 30
pipeline = (br >> indexed_epic.reranker())
pt.Experiment(
    [br, pipeline],
    dataset.get_topics('description'),
    dataset.get_qrels(),
    names=['DPH', 'DPH >> EPIC (indexed)'],
    eval_metrics=["map", "recip_rank", "ndcg", "ndcg_cut.10", "mrt"]
)

## Tuning re-ranking threshold

[Prior work suggests](https://arxiv.org/pdf/1904.12683.pdf) that the re-ranking cutoff threshold is an important model hyperparameter. Let's see how this parameter affects EPIC.

In [ ]:
cutoffs = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
res = pt.Experiment(
    [pt.BatchRetrieve(index) % cutoff >> indexed_epic.reranker() for cutoff in cutoffs],
    dataset.get_topics('description'),
    dataset.get_qrels(),
    names=[f'c={cutoff}' for cutoff in cutoffs],
    eval_metrics=["map", "recip_rank", "ndcg", "ndcg_cut.10", "mrt"]
)
res

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.plot(res['name'], res['ndcg_cut_10'], label='ndcg_cut_10')
plt.plot(res['name'], res['recip_rank'], label='recip_rank')
plt.ylabel('ndcg_cut_10 / recip_rank')
plt.legend()
plt.show()
plt.clf()
plt.plot(res['name'], res['mrt'])
plt.ylabel('mrt')
plt.show()

It appears that the optimal re-ranking threshold for this collection is around 40-50. This also avoids excessive re-ranking time, which grows roughly linearly with larger thredhols. In pratice, this paramter should be tuned on a held-out validation set to avoid over-fitting.

## monoT5

The [monoT5](https://arxiv.org/abs/2003.06713) model scores documents using a causal language model. Let's see how this approach works on TREC COVID.

The `MonoT5ReRanker` class from `pyterrier_t5` automatically loads a version of the monoT5 ranker that is trained on the MS MARCO passage dataset.

In [ ]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')

In [ ]:
br = pt.BatchRetrieve(index) % 30
pipeline = (br >> pt.text.get_text(dataset, 'abstract') >> monoT5)
pt.Experiment(
    [br, pipeline],
    dataset.get_topics('description'),
    dataset.get_qrels(),
    names=['DPH', 'DPH >> T5'],
    eval_metrics=["map", "recip_rank", "ndcg", "ndcg_cut.10", "mrt"]
)

#  That's all folks
We have a separate notebook in Part 3 for ColBERT, as ColBERT needs a different version of Huggingface Trasformers. Refer to the [tutorial repostiory](https://github.com/terrier-org/ecir2021tutorial) for the other Part 3 notebook.

If you arent coming back for Part 4 of the tutorial, please dont forget to complete our exit quiz: https://forms.office.com/r/2WbpLiQmWV